## Climate baseline

In [2]:
import os
import xarray as xr
import pandas as pd
import geopandas as gpd
import xesmf as xe
import regionmask
import xarray_regrid # type: ignore

# when using "rename", dont change the target variable names (of course, change the original variable names according your files)

In [3]:
# os.chdir('/home/rooda/Dropbox/Patagonia/')
os.chdir('/workspaces/oggm_redo/rodrigo_data')

# the period
period = slice("1980-01-01", "2019-12-31")

In [7]:
# sample file (to be shared)
# example    = xr.open_dataset("example_nc.nc")
example    = xr.open_dataset("histalp_merged_hef.nc")

# your DEM close to year 2000
# dem        = xr.open_dataset("file_dem.tif").rename({'y':'lat', 'x':'lon', 'band_data':'hgt'})
dem        = xr.open_dataset("srtm_dem_s37_w072_1arc_v3.tif").rename({'y':'lat', 'x':'lon', 'band_data':'hgt'})
# ^ my SRTM 2000 DEM for the area 
dem        = dem.sel(band=1, drop=True).drop("spatial_ref").fillna(0)
dem.hgt.attrs['units'] = "m"


/tmp/ipykernel_49256/4036720098.py:8: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  dem        = dem.sel(band=1, drop=True).drop("spatial_ref").fillna(0)


In [8]:
# monthly cr2met
pp_cr2met   = xr.open_dataset("pp_cr2met.nc",  chunks = "auto").sel(time = period)
t2m_cr2met  = xr.open_dataset("t2m_cr2met.nc",  chunks = "auto").sel(time = period)

FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/oggm_redo/rodrigo_data/pp_cr2met.nc'

In [ ]:

# regrid and merge
regridder   = xe.Regridder(dem, pp_cr2met, "bilinear")
dem_cr2met  = regridder(dem)

# merge all variables
cr2met = xr.merge([pp_cr2met, t2m_cr2met, dem_cr2met]).rename({'pp':'prcp', 't2m':'temp'})
cr2met = cr2met.where((cr2met.lon >= -76) & (cr2met.lon <= -68) & (cr2met.lat >= -56) & (cr2met.lat <= -40), drop=True)
cr2met = cr2met[["lon", "lat", "time", "prcp", "temp", "hgt"]]

In [ ]:
# unit attribute
cr2met.prcp.attrs['units'] = "mm"
cr2met.temp.attrs['units'] = "degc"
cr2met.hgt.attrs['units'] = "m"

# standard_name attribute
cr2met.prcp.attrs['standard_name'] = "precipitation"
cr2met.temp.attrs['standard_name'] = "temperature"

In [ ]:
# save me
cr2met.to_netcdf("climate_to_use.nc")